In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# display plots in this notebook
%matplotlib inline

# for store the results
import pickle
import gzip

# for include pycode
import sys
import os
sys.path.insert(0,'../pycode')

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
caffe_root = '/opt/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
from caffe import layers as L
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.


import aestheticNet
from preprocess import utilities 

# Generacion de train y test
Primero creamos dos txt con la informacion del fichero y de su clase

In [3]:
data = pickle.load(gzip.open('../packages/AVA_info.pklz','rb',2))

In [4]:
data.loc[:,'id'] = data['id'].apply(str)
classes = np.array(data.sort_values(['id']).loc[:,'Class'])
ids = np.array(data.sort_values(['id']).loc[:,'id'])
aux_list =  np.array(['/home/frubio/AVA/AVADataset/{:}.jpg'.format(i) for i in ids])

In [5]:
np.random.seed(1000)
random_idx = np.random.randint(3, size=len(ids))

In [6]:
train_files = aux_list[random_idx <= 1]
train_classes = classes[random_idx <= 1]
# Class balanced
new_indexes = utilities.balance_class(train_classes)
train_classes = train_classes[new_indexes]
train_files = train_files[new_indexes]

test_files = aux_list[random_idx == 2]
test_classes = classes[random_idx == 2]

In [7]:
df_train = pd.DataFrame(list(zip(train_files,train_classes)))
df_train.columns = ['image_filename', 'label']
df_train.to_csv('../../AVA/train_partition_finetuning.txt', sep=' ', header=None, index=None)

df_test = pd.DataFrame(list(zip(test_files,test_classes)))
df_test.columns = ['image_filename', 'label']
df_test.to_csv('../../AVA/test_partition_finetuning.txt', sep=' ', header=None, index=None)

# Aqui comienza la chicha

In [8]:
caffe.set_device(0)  # if we have multiple GPUs, pick the first one
#caffe.set_mode_cpu()

In [9]:
weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

In [10]:
niter = 100000  # number of iterations to train

# Reset style_solver as before.
style_solver_filename = aestheticNet.solver(aestheticNet.aest_net(train=True,caffe_aes=True))
style_solver = caffe.get_solver(style_solver_filename)
style_solver.net.copy_from(weights)

In [ ]:
print ('Running solvers for %d iterations...' % niter)
solvers = [('pretrained', style_solver)]
loss, acc, weights = aestheticNet.run_solvers(niter, solvers)
print ('Done.')

train_loss = loss['pretrained']
train_acc = acc['pretrained']
style_weights = weights['pretrained']

# Delete solvers to save memory.
del style_solver, solvers

Running solvers for 200 iterations...
  0) pretrained: loss=0.693, acc=32%
 10) pretrained: loss=0.684, acc=60%
 20) pretrained: loss=0.651, acc=60%
 30) pretrained: loss=0.697, acc=54%
 40) pretrained: loss=0.655, acc=62%
 50) pretrained: loss=0.704, acc=46%
 60) pretrained: loss=0.560, acc=76%
 70) pretrained: loss=0.675, acc=60%
 80) pretrained: loss=0.757, acc=50%
 90) pretrained: loss=0.779, acc=44%
100) pretrained: loss=0.728, acc=42%
110) pretrained: loss=0.667, acc=60%
120) pretrained: loss=0.648, acc=62%
130) pretrained: loss=0.657, acc=66%
140) pretrained: loss=0.646, acc=62%
150) pretrained: loss=0.778, acc=48%
160) pretrained: loss=0.644, acc=64%
170) pretrained: loss=0.687, acc=70%
180) pretrained: loss=0.714, acc=54%


In [ ]:
os.rename(weights['pretrained'], "/home/frubio/AVA/aesthetic_finetuning.caffemodel")